### Example: Helixes
Using the same system setup as for the ideal chain, we setup a system of non-interacting polymer chains, run a short simulation, and examine the resulting trajectory.

In [3]:
import h5py
import numpy as np
from scipy.spatial.transform import Rotation as R


def setup_system(box_size, n_molecules, chain_length, out_path, bond_length=0.5):
    """Setup a simple system of polymer chains
    
    Parameters
    ----------
    box_size : list of float
        Simulation box size in nm.
    n_molecules : int
        Total number of molecules of `chain_length` monomer units.
    chain_length : int
        Length of each polymer chain in number of monomers.
    out_path : str
        Path of the created structure file.
    bond_length : float, optional
        Length of intramolecular bonds in the chains.
    """
    box_size = np.asarray(box_size, dtype=np.float32)
    n_particles = n_molecules * chain_length
    
    with h5py.File(out_path, "w") as out_file:
        position_dataset = out_file.create_dataset(
            "coordinates",
            (1, n_particles, box_size.size,),
            dtype="float32",
        )
        types_dataset = out_file.create_dataset(
            "types",
            (n_particles,),
            dtype="i",
        )
        names_dataset = out_file.create_dataset(
            "names",
            (n_particles,),
            dtype="S10",
        )
        indices_dataset = out_file.create_dataset(
            "indices",
            (n_particles,),
            dtype="i",
        )
        molecules_dataset = out_file.create_dataset(
            "molecules",
            (n_particles,),
            dtype="i",
        )
        bonds_dataset = out_file.create_dataset(
            "bonds",
            (n_particles, 3,),
            dtype="i",
        )
    
        head_positions = np.random.uniform(
            low=np.zeros_like(box_size),
            high=box_size,
            size=(n_molecules, box_size.size,),
        )
        
        # Randomly coiled chains
        for i in range(n_molecules):
            position_dataset[0, i*chain_length, :] = head_positions[i, :]
            for j in range(1, chain_length):
                position_dataset[0, i*chain_length + j, :] = (
                    head_positions[i, :] 
                    + R.random().as_matrix().dot(np.array([
                        bond_length, 0.0, 0.0,
                    ]))
                )
                            
        types_dataset[...] = 0
        names_dataset[...] = np.string_("A")
        indices_dataset[...] = np.arange(n_particles)
    
        # molecules
        molecules_dataset[...] = np.repeat(np.arange(n_molecules), chain_length)
        
        # bonds
        bonds_dataset[...] = -1
        bonds_dataset[:, 0] = np.arange(n_particles) - 1
        bonds_dataset[::chain_length, 0] = -1
        
        bonds_dataset[:, 1] = np.arange(n_particles) + 1
        bonds_dataset[chain_length-1::chain_length, 1] = -1
# ==============================================================================

# 10 chains in a 30nm x 30nm x 30nm simulation box.
setup_system([30, 30, 30], 15, 30, "helixes.HDF5", bond_length=0.31)

### Run the simulation
Disable the field interactions with `--disable-field`.

```bash
mpirun -n 6 python3 /Users/mortenledum/Documents/HyMD/hymd/__main__.py \
                    helixes.toml      \
                    helixes.HDF5      \
                    --disable-field   \
                    --seed 1          \
                    --verbose 1
```